In [4]:
import pandas as pd
import datetime
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
df_stops = pd.read_csv('stops.txt')
df_routes = pd.read_csv('routes.txt')
df_trips = pd.read_csv('trips.txt')
df_stop_times = pd.read_csv('stop_times.txt')
df_routes = pd.read_csv('routes.txt')

In [6]:
def edges_build(stop_ids_list):
    l = len(stop_ids_list)-1
    i = 0
    edges_list = list()
    while (i<l):
        edges_list.append((stop_ids_list[i], stop_ids_list[i+1]))
        i = i+1
    return edges_list

def route_id_to_name(route_id):
    return df_routes[df_routes["route_id"] == route_id]["route_short_name"].tolist()[0]

def stop_to_route(stop_name):
    stop_id = stop_name_to_id(stop_name)["stop_id"].tolist()[0]
    trip_id = df_stop_times[df_stop_times["stop_id"] == int(stop_id)]["trip_id"].tolist()[0]
    route_id = df_trips[df_trips["trip_id"] == trip_id]["route_id"].tolist()[0]
    return route_id_to_name(route_id)

def stop_id_to_name(stop_id):
    if (type(stop_id) == str):
        return df_stops[df_stops["stop_id"] == stop_id]["stop_name"].tolist()[0]
    return df_stops[df_stops["stop_id"] == str(stop_id)]["stop_name"].tolist()[0]

def stop_name_to_id(stop_name):
        return df_stops[df_stops["stop_name"] == stop_name]

In [7]:
def belongs_to_list (list_, x):
    for elt in list_:
        if (x == elt):
            return True
    return False


def belongs_to_dict (dict_, x):
    return_dict = dict()
    for key, list_ in dict_.items():
         if (belongs_to_list(list_, x)):
                #lists.append(list_)
             return_dict[key] = list_
    return return_dict

#print("À l'indice {} se trouve {}.".format(i, elt))

def breakdown_affected_stops (dict_, stop_1, stop_2):
    affected_stops = set()
    for key, list_ in dict_.items():
        if (belongs_to_list(list_, stop_1) & belongs_to_list(list_, stop_2)):
            index_1 = list_.index(stop_1)
            index_2 = list_.index(stop_2)
            for index, elt in enumerate(list_):
                if ((index < index_2) & (index > index_1) | (index < index_1) & (index > index_2)):
                    affected_stops.add(elt)
    return affected_stops                   
     
            

In [8]:
# suppression des repetitions dans trips et stop_times
df_trips = df_trips.drop_duplicates(["route_id", "direction_id", "trip_headsign"] )
df_stop_times = df_stop_times[df_stop_times["trip_id"].isin(df_trips["trip_id"])]

In [9]:
paths_dict = dict()
keys_info = dict ()
for elt in df_trips["trip_id"]:
    route = df_trips[df_trips["trip_id"] == elt]["route_id"].tolist()[0]
    direction = df_trips[df_trips["trip_id"] == elt]["direction_id"].tolist()[0]
    trip_headsign = df_trips[df_trips["trip_id"] == elt]["trip_headsign"].tolist()[0]
    info = [route, direction, trip_headsign]
    df = df_stop_times[df_stop_times["trip_id"] == elt]
    paths_dict[elt] = df["stop_id"].tolist()  
    keys_info[elt] = info


In [10]:
belongs_to_dict(paths_dict, 7438)["10611428-2016-TFEV_C-Dimanche-06"][0]
#stop_id_to_name (belongs_to_dict(paths_dict, 7438)["10611428-2016-TFEV_C-Dimanche-06"][0])

7440

In [11]:
stop_name_to_id("Bergonié")

,stop_id,stop_name,stop_lat,stop_lon,stop_desc,zone_id,stop_url,stop_code,location_type,parent_station
61,3749,Bergonié,44.825031,-0.578216,NaN,33063,NaN,BERGA,0,BERGO
62,3750,Bergonié,44.825020,-0.578278,NaN,33063,NaN,BERGR,0,BERGO
248,BERGO,Bergonié,44.825025,-0.578247,NaN,33063,NaN,BERGO,1,NaN


In [9]:
stop_name_to_id("Forum")

,stop_id,stop_name,stop_lat,stop_lon,stop_desc,zone_id,stop_url,stop_code,location_type,parent_station
55,3743,Forum,44.811948,-0.590942,NaN,33522,NaN,FORUA,0,FORUM
56,3744,Forum,44.811965,-0.590981,NaN,33522,NaN,FORUR,0,FORUM
245,FORUM,Forum,44.811956,-0.590962,NaN,33522,NaN,FORUM,1,NaN


In [10]:
breakdown_affected_stops(paths_dict, 3749, 3744)
# ensemble vide car il faut préciser l'id de la bonne direction

set()

In [11]:
breakdown_affected_stops(paths_dict, 3749, 3743)
# renvoie les ids de roustaing et barriere st-genes

{3745, 3747}

In [13]:
stop_id_to_name("3747")

'Barri\xc3\xa8re Saint-Gen\xc3\xa8s'

In [17]:
stop_id_to_name("3745")

'Roustaing'